In [1]:
catelog = {
  "category": ("Dresses","tops","bottoms","winterwear","jeans","Tröjor & koftor","Kjolar","shorts","sportklader","jackor","swimwear","nightwear","Panties & Bra","socks","acessories"
              "shoes"),
    
  "main_urls": ("https://www.bonprix.se/kategori/barn-flickor-klaenningar/","https://www.bonprix.se/kategori/barn-flickor-skjortor/",
              "https://www.bonprix.se/kategori/barn-flickor-byxor/","https://www.bonprix.se/kategori/barn-flickor-jeans/","https://www.bonprix.se/kategori/barn-flickor-troejor-koftor/",
              "https://www.bonprix.se/kategori/barn-flickor-troejor-koftor/","https://www.bonprix.se/kategori/barn-flickor-kjolar/",
              "https://www.bonprix.se/kategori/barn-flickor-shorts/","https://www.bonprix.se/kategori/barn-flickor-traeningsklaeder/",
              "https://www.bonprix.se/kategori/barn-flickor-jackor/","https://www.bonprix.se/kategori/barn-flickor-badklaeder/",
               "https://www.bonprix.se/kategori/barn-underklaeder-flickor-nattklaeder/","https://www.bonprix.se/kategori/barn-underklaeder-flickor-trosor/",
                "https://www.bonprix.se/kategori/barn-underklaeder-flickor-strumpor/","https://www.bonprix.se/kategori/barn-flickor-skor/",
               "https://www.bonprix.se/kategori/barn-flickor-accessoarer/")
    
  }

gender='girl'

In [2]:
import csv


f = open("bonprix.csv", "w", encoding='utf-8')
writer = csv.DictWriter(f, fieldnames=["name","brand","gender","category","size","price","color","composition","url","img_url","article_no","blob_url"])
writer.writeheader()
f.close()

import pandas as pd
df=pd.read_csv("bonprix.csv", encoding='utf-8')
df.head()

,name,brand,gender,category,size,price,color,composition,url,img_url,article_no,blob_url


In [3]:
!mkdir bonprix

In [4]:
!pip3 install gcloud
!pip3 install google-cloud-storage
!pip3 show requests
!python -m pip install google-cloud

Name: requests
Version: 2.22.0
Summary: Python HTTP for Humans.
Home-page: http://python-requests.org
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache 2.0
Location: /usr/local/lib/python3.5/dist-packages
Requires: urllib3, certifi, chardet, idna


In [5]:
!pip3 install google-cloud

In [6]:
!pip install google-cloud-storage

In [7]:
!pip install virtualenv



In [8]:
!virtualenv hello

Using base prefix '/usr'
New python executable in /home/jupyter/hello/bin/python3
Also creating executable in /home/jupyter/hello/bin/python
Installing setuptools, pip, wheel...
done.


In [9]:
!virtualenv hello\Scripts\activate 
!virtualenv hello\Scripts\pip.exe install google-cloud-storage

Using base prefix '/usr'
New python executable in /home/jupyter/helloScriptsactivate/bin/python3
Also creating executable in /home/jupyter/helloScriptsactivate/bin/python
Installing setuptools, pip, wheel...
done.
There must be only one argument: DEST_DIR (you gave helloScriptspip.exe install google-cloud-storage)
Usage: virtualenv [OPTIONS] DEST_DIR

Options:
  --version             show program's version number and exit
  -h, --help            show this help message and exit
  -v, --verbose         Increase verbosity.
  -q, --quiet           Decrease verbosity.
  -p PYTHON_EXE, --python=PYTHON_EXE
                        The Python interpreter to use, e.g.,
                        --python=python3.5 will use the python3.5 interpreter
                        to create the new environment.  The default is the
                        interpreter that virtualenv was installed with
                        (/usr/bin/python3)
  --clear               Clear out the non-root install and start 

In [10]:

from google.cloud import storage

def upload_to_bucket(blob_name, path_to_file):
    """ Upload data to a bucket""" #doc string in function

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'auth.json')

    #print(buckets = list(storage_client.list_buckets())

    bucket = storage_client.get_bucket('urbankiddo')
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)

    #returns a public url
    return blob.public_url

In [11]:
import requests
import time
from random import randint
from bs4 import BeautifulSoup

import urllib.request
import csv

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.80 Safari/537.36'}

for c in range(len(catelog['category'])):
    category=catelog['category'][c]
    su=catelog['main_urls'][c]
    
    sub_urls=[su]

    rr =requests.get(su, headers=headers)
    html_doc0=rr.content.decode()
    soup0 = BeautifulSoup(html_doc0, 'html.parser')

    pages=soup0.find_all("li",attrs={"class":"product-pager-page"})
    for pg in pages:
        try:
            sub_urls.append(pg.a['href'])
        except:
            continue
    print(sub_urls)
    
    
    for page in sub_urls:
    
        r =requests.get(page, headers=headers)
        html_doc=r.content.decode()
        soup = BeautifulSoup(html_doc, 'html.parser')

        products=soup.find_all("div", attrs={ "class" :"product-image"})
        for p in products:
            url=p.a['href']
            article_no=p.a['data-order-number-with-promo']

            
            s =requests.get(url, headers=headers)
            html_doc2=s.content.decode()

    #         /produkt/947444.json

            soup2 = BeautifulSoup(html_doc2, 'html.parser')
            name=soup2.find("h1", attrs={ "class" :"product-name"}).text





            price=soup2.find("span", attrs={ "class" :"price-tag"}).text
            info=soup2.find("div", attrs={ "class" :"product-attributes"})

            p=info.find_all("span",attrs={"class":"product-attribute-value glossarable"})
            try:
                composition=p[0].text.strip()
            except:
                composition=None
            try:
                color=p[1].text.strip()
            except:
                color=None
            
            
            img_url='https:'+soup2.find("div", attrs={ "class" :"image-container"}).img['data-product-image-src']



            ac='bonprix'+'/'+article_no+str('.jpg')
    #             opener = urllib.request.URLopener()
    #             opener.addheader('User-Agent', 'whatever')
    #             filename, headers2 = opener.retrieve(img_url, ac)

            rr = requests.get(img_url, allow_redirects=True)
            open(ac, 'wb').write(rr.content)

#             blob_name=str('bonprix')+str('/')+str(article_no)
#             path_to_file='bonprix/'+article_no+str('.jpg')
#             blob_url=upload_to_bucket(blob_name, path_to_file)
            
            

            brand=soup2.find("span", attrs={ "class" :"product-brand"}).span['content']
            sizes=soup2.find("ul",attrs={"id":"product-variant-size"})
            for ss in sizes.find_all("li"):
                size=ss.a.text
                with open("bonprix.csv", "a", encoding='utf-8') as f:
                                writer = csv.writer(f)
                                writer.writerow([name,brand,gender,category,size,price,color,composition,url,img_url,article_no])

    #             print(url,name,img_url,price,gender,composition,color,article_no,brand,size)


        
        
        
        

['https://www.bonprix.se/kategori/barn-flickor-klaenningar/', 'https://www.bonprix.se/kategori/barn-flickor-klaenningar/?page=2']
['https://www.bonprix.se/kategori/barn-flickor-skjortor/']
['https://www.bonprix.se/kategori/barn-flickor-byxor/']
['https://www.bonprix.se/kategori/barn-flickor-jeans/']
['https://www.bonprix.se/kategori/barn-flickor-troejor-koftor/']
['https://www.bonprix.se/kategori/barn-flickor-troejor-koftor/']
['https://www.bonprix.se/kategori/barn-flickor-kjolar/']
['https://www.bonprix.se/kategori/barn-flickor-shorts/']
['https://www.bonprix.se/kategori/barn-flickor-traeningsklaeder/']
['https://www.bonprix.se/kategori/barn-flickor-jackor/']
['https://www.bonprix.se/kategori/barn-flickor-badklaeder/']
['https://www.bonprix.se/kategori/barn-underklaeder-flickor-nattklaeder/']
['https://www.bonprix.se/kategori/barn-underklaeder-flickor-trosor/']
['https://www.bonprix.se/kategori/barn-underklaeder-flickor-strumpor/']
['https://www.bonprix.se/kategori/barn-flickor-skor/'

In [12]:
url

'https://www.bonprix.se/produkt/vinterstoevel-gammaldags-rosa-974172/?shop=429072#image'

In [14]:
catelog = {
  "category": ("tshirts","byxor","trojor and koftor","shorts","jeans","jackor","sports klader","stora klader","badbyxor","underpants","pyjamas","socks","shoes"),
    
  "main_urls": ("https://www.bonprix.se/kategori/barn-pojkar-t-shirts/","https://www.bonprix.se/kategori/barn-pojkar-byxor/",
                "https://www.bonprix.se/kategori/barn-pojkar-troejor-koftor/","https://www.bonprix.se/kategori/barn-pojkar-shorts/",
               "https://www.bonprix.se/kategori/barn-pojkar-jeans/","https://www.bonprix.se/kategori/barn-pojkar-jackor/",
               "https://www.bonprix.se/kategori/barn-pojkar-sportmode/","https://www.bonprix.se/kategori/barn-pojkar-stora-storlekar/",
                "https://www.bonprix.se/kategori/barn-pojkar-badbyxor/","https://www.bonprix.se/kategori/barn-underklaeder-pojkar-kalsonger/",
               "https://www.bonprix.se/kategori/barn-underklaeder-pojkar-pyjamas/","https://www.bonprix.se/kategori/barn-underklaeder-pojkar-sockor/",
               "https://www.bonprix.se/kategori/barn-pojkar-skor/")
    
  }

gender='boy'

In [15]:
import requests
import time
from random import randint
from bs4 import BeautifulSoup

import urllib.request
import csv

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.80 Safari/537.36'}

for c in range(len(catelog['category'])):
    category=catelog['category'][c]
    su=catelog['main_urls'][c]
    
    sub_urls=[su]

    rr =requests.get(su, headers=headers)
    html_doc0=rr.content.decode()
    soup0 = BeautifulSoup(html_doc0, 'html.parser')

    pages=soup0.find_all("li",attrs={"class":"product-pager-page"})
    for pg in pages:
        try:
            sub_urls.append(pg.a['href'])
        except:
            continue
    print(sub_urls)
    
    
    for page in sub_urls:
    
        r =requests.get(page, headers=headers)
        html_doc=r.content.decode()
        soup = BeautifulSoup(html_doc, 'html.parser')

        products=soup.find_all("div", attrs={ "class" :"product-image"})
        for p in products:
            url=p.a['href']
            
            article_no=p.a['data-order-number-with-promo']
            
            s =requests.get(url, headers=headers)
            html_doc2=s.content.decode()

    #         /produkt/947444.json

            soup2 = BeautifulSoup(html_doc2, 'html.parser')
            name=soup2.find("h1", attrs={ "class" :"product-name"}).text

         




            price=soup2.find("span", attrs={ "class" :"price-tag"}).text
            info=soup2.find("div", attrs={ "class" :"product-attributes"})

            p=info.find_all("span",attrs={"class":"product-attribute-value glossarable"})

            try:
                composition=p[0].text.strip()
            except:
                composition=None
            try:
                color=p[1].text.strip()
            except:
                color=None
            
            img_url='https:'+soup2.find("div", attrs={ "class" :"image-container"}).img['data-product-image-src']



            ac='bonprix'+'/'+article_no+str('.jpg')
    #             opener = urllib.request.URLopener()
    #             opener.addheader('User-Agent', 'whatever')
    #             filename, headers2 = opener.retrieve(img_url, ac)

            rr = requests.get(img_url, allow_redirects=True)
            open(ac, 'wb').write(rr.content)

#             blob_name=str('bonprix')+str('/')+str(article_no)
#             path_to_file='bonprix/'+article_no+str('.jpg')
#             blob_url=upload_to_bucket(blob_name, path_to_file)
            
            

            brand=soup2.find("span", attrs={ "class" :"product-brand"}).span['content']
            sizes=soup2.find("ul",attrs={"id":"product-variant-size"})
            for ss in sizes.find_all("li"):
                size=ss.a.text
                with open("bonprix.csv", "a", encoding='utf-8') as f:
                                writer = csv.writer(f)
                                writer.writerow([name,brand,gender,category,size,price,color,composition,url,img_url,article_no])

    #             print(url,name,img_url,price,gender,composition,color,article_no,brand,size)


        
        
        
        

['https://www.bonprix.se/kategori/barn-pojkar-t-shirts/']
['https://www.bonprix.se/kategori/barn-pojkar-byxor/']
['https://www.bonprix.se/kategori/barn-pojkar-troejor-koftor/']
['https://www.bonprix.se/kategori/barn-pojkar-shorts/']
['https://www.bonprix.se/kategori/barn-pojkar-jeans/']
['https://www.bonprix.se/kategori/barn-pojkar-jackor/']
['https://www.bonprix.se/kategori/barn-pojkar-sportmode/']
['https://www.bonprix.se/kategori/barn-pojkar-stora-storlekar/']
['https://www.bonprix.se/kategori/barn-pojkar-badbyxor/']
['https://www.bonprix.se/kategori/barn-underklaeder-pojkar-kalsonger/']
['https://www.bonprix.se/kategori/barn-underklaeder-pojkar-pyjamas/']
['https://www.bonprix.se/kategori/barn-underklaeder-pojkar-sockor/']
['https://www.bonprix.se/kategori/barn-pojkar-skor/']


In [16]:
su=catelog['main_urls'][0]
sub_urls=[su]

r =requests.get(su, headers=headers)
html_doc=r.content.decode()
soup = BeautifulSoup(html_doc, 'html.parser')

pages=soup.find_all("li",attrs={"class":"product-pager-page"})
for pg in pages:
    try:
        sub_urls.append(pg.a['href'])
    except:
        continue
print(sub_urls)

['https://www.bonprix.se/kategori/barn-pojkar-t-shirts/']


In [17]:
pages

[]

In [18]:
p=info.find_all("span",attrs={"class":"product-attribute-value glossarable"})
composition=p[0].text.strip()
color=p[1].text.strip()
article_no=p[2].text.strip()
print(composition,color,article_no)

Yttermaterial: skinnimitation, Sport sula: syntetmaterial, Innersula: textil jeansblå Låg


In [19]:
info

<div class="product-attributes">
<p class="product-attribute clearfix"><strong class="product-attribute-name fromVelocity" id="ds_bulletpoint_0">Material:</strong><span class="product-attribute-value glossarable">
		            		                Yttermaterial: skinnimitation, Sport sula: syntetmaterial, Innersula: textil
		            		        </span></p>
<p class="product-attribute clearfix"><strong class="product-attribute-name fromVelocity" id="ds_bulletpoint_1">Färg:</strong><span class="product-attribute-value glossarable">
		            		                jeansblå
		            		        </span></p>
<p class="product-attribute clearfix"><strong class="product-attribute-name fromVelocity" id="ds_bulletpoint_2">Klacktyp:</strong><span class="product-attribute-value glossarable">
		            		                Låg
		            		        </span></p>
<p class="product-attribute clearfix"><strong class="product-attribute-name fromVelocity" id="ds_bulletpoint_3">Klackhöjd:</strong><sp

In [20]:
'https:'+img_url

'https:https://image01.bonprix.se/assets/460x644/1548838391/19021671-87VfUnrD.jpg'

In [21]:
soup2.find("div", attrs={ "class" :"image-container"})

<div class="image-container">
<meta content="https://www.bonprix.se//image01.bonprix.se/assets/460x644/1548838391/19021671-87VfUnrD.jpg" itemprop="image">
<img alt="Snörkänga, bpc bonprix collection" data-height="644" data-main-image="//image01.bonprix.se/assets/460x644/1548838391/19021671-87VfUnrD.jpg" data-main-image-2x="//image01.bonprix.se/assets/460x644/2x/1548838391/19021671-87VfUnrD.jpg" data-product-image-src="//image01.bonprix.se/assets/460x644/1548838391/19021671-87VfUnrD.jpg" data-product-image-src-2x="//image01.bonprix.se/assets/460x644/2x/1548838391/19021671-87VfUnrD.jpg" data-product-zoom-image-src="//image01.bonprix.se/assets/1400x1960/1548838391/19021671-87VfUnrD.jpg" data-width="460" data-zoom-image-src="//image01.bonprix.se/assets/1400x1960/1548838391/19021671-87VfUnrD.jpg" data-zoom-text="Klicka för att förstora" id="productimage" src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAUAAAAHAQAAAADo3LpDAAAAAnRSTlMAAQGU/a4AAAALSURBVHgBY/yBDAEvvgbQiftLAwAAAABJRU5ErkJggg=